# Import Necessary Libraries

In [1]:
!pip install --upgrade transformers
!pip install --upgrade bert-extractive-summarizer
!pip install --upgrade spacy
!pip install fake-useragent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.


In [2]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.doiTDP7qRT/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.QQF0PHj588/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.lKfdUVX2p7/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Webscraping from moneycontrol.com

In [4]:
class MoneyControlScraper:

  def __init__(self):
      pass

  # Function to click the "Read More" button using Selenium
  def click_read_more(self,driver, url):
      driver.get(url)
      try:
          read_more_button = WebDriverWait(driver, 10).until(
              EC.presence_of_element_located((By.XPATH, "//button[contains(text(),'Read More')]"))
          )
          read_more_button.click()
      except:
          pass

  def get_article_links(self,url):
      response = requests.get(url)
      article_links = []

      if response.status_code == 200:
          soup = BeautifulSoup(response.text, 'html.parser')
          h2_elements = soup.find_all('h2')

          for h2_element in h2_elements:
              a_tag = h2_element.find('a')
              if a_tag:
                  link = a_tag['href']
                  article_links.append(link)

      return article_links

  def scrape_article_text(self,article_link):
      chromeOptions = webdriver.ChromeOptions()
      chromeOptions.add_argument("--no-sandbox")
      chromeOptions.add_argument("--headless")
      chromeOptions.add_argument("--disable-setuid-sandbox")
      chromeOptions.add_argument("--remote-debugging-port=9222")
      chromeOptions.add_argument("--disable-dev-shm-using")
      chromeOptions.add_argument("--disable-dev-shm-uage")
      chromeOptions.add_argument("--disable-extensions")
      chromeOptions.add_argument("--disable-gpu")
      chromeOptions.add_argument("start-maximized")
      chromeOptions.add_argument("disable-infobars")
      chromeOptions.add_argument(r"user-data-dir=.\cookies\\test")

      driver = webdriver.Chrome(options=chromeOptions)
      self.click_read_more(driver, article_link)

      if driver.page_source:
          article_soup = BeautifulSoup(driver.page_source, 'html.parser')
          div_elements = article_soup.find_all('div', class_='page_left_wrapper')  # Adjust class name as needed
          article_texts = []

          for div_element in div_elements:
              p_tags = div_element.find_all('p')
              article_text = '\n'.join(p.text for p in p_tags)
              article_texts.append(article_text)

          driver.quit()
          return article_texts
      else:
          return []

In [6]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

# URL of the page you want to scrape
url = "https://www.crunchbase.com/organization/boat/company_overview/overview_timeline"

# Set up the Selenium WebDriver with a headless browser (e.g., Chrome)
chromeOptions = webdriver.ChromeOptions()
ua = UserAgent()
user_agent = ua.random
chromeOptions.add_argument("--no-sandbox")
chromeOptions.add_argument("--headless")
chromeOptions.add_argument("--disable-setuid-sandbox")
chromeOptions.add_argument("--remote-debugging-port=9222")
chromeOptions.add_argument("--disable-dev-shm-using")
chromeOptions.add_argument("--disable-dev-shm-uage")
chromeOptions.add_argument("--disable-extensions")
chromeOptions.add_argument("--disable-gpu")
chromeOptions.add_argument("start-maximized")
chromeOptions.add_argument("disable-infobars")
chromeOptions.add_argument(r"user-data-dir=.\cookies\\test")
chromeOptions.add_argument("window-size=100,100")
chromeOptions.add_argument(f'--user-agent={user_agent}')
driver = webdriver.Chrome(options=chromeOptions)

# Navigate to the URL
driver.get(url)
# print(driver.page_source)

# Wait for the page to load (adjust the timeout as needed)
timeout = 20
try:
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'section-content-wrapper'))
    WebDriverWait(driver, timeout).until(element_present)
except Exception as e:
    print("Exception:",e)

# Give the page some additional time to ensure all content is loaded
time.sleep(10)

# Get the page source after waiting
page_source = driver.page_source

# Close the browser
driver.quit()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find all 'a' tags inside 'div' tags with class 'activity-row ng-star-inserted'
activity_rows = soup.find_all('div', class_='activity-row ng-star-inserted')
for row in activity_rows:
    a_tags = row.find_all('a')
    for a_tag in a_tags:
        print(a_tag['href'])  # Replace with the attribute you want to extract

# You can now process the 'a' tags as needed

https://bwdisrupt.businessworld.in/article/BoAt-Records-Rs3-400cr-Revenue-In-FY23-Report-/13-11-2023-498478
https://economictimes.indiatimes.com/industry/cons-products/electronics/electronics-industry-needs-china-help-now-for-design-engineering-but-will-become-self-reliant-boat-ceo/articleshow/104643210.cms
https://www.gadgetsnow.com/wearables/boat-launches-airdopes-flex-454-earbuds-price-features-and-more/articleshow/103833982.cms
https://inc42.com/buzz/70-boat-products-made-india-cofounder-aman-gupta/
https://www.gadgetsnow.com/wearables/boat-launches-lunar-orb-smartwatch-price-specs-and-more/articleshow/103204903.cms
https://www.digit.in/news/wearable-devices/boat-smart-ring-specifications-tipped-68790.html
https://www.nextpit.com/boat-smart-ring-presents-itself-with-fitness-tracker-features
https://www.digit.in/news/wearable-devices/boat-smart-ring-wearable-device-68516.html
https://economictimes.indiatimes.com/markets/ipos/fpos/boat-looking-at-fy25-fy26-timeframe-for-ipo-says-co-f

# Text Summarization using Pre-Trained Model

In [9]:
from summarizer import Summarizer,TransformerSummarizer
import warnings
warnings.filterwarnings("ignore")

url = "https://www.moneycontrol.com/news/tags/sugar-cosmetics.html"
moneyControlScraperObject = MoneyControlScraper()
article_links = moneyControlScraperObject.get_article_links(url)
artText = ""
for link in article_links[:5]:
    try:
      article_texts = moneyControlScraperObject.scrape_article_text(link)
      artText = "\n".join(article_texts)
      GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
      full = ''.join(GPT2_model(artText, min_length=60))
      print("Link:", link)
      print("\nSummary:\n")
      print(full)
      print("\nFull Text Article:\n")
      print(artText)
      print("\n---\n")
    except Exception as e:
      print("Exception occurred: ",e)

(…).co/gpt2-medium/resolve/main/config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

(…)e.co/gpt2-medium/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)e.co/gpt2-medium/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)/gpt2-medium/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Link: https://www.moneycontrol.com/news/trends/vineeta-singh-recalls-sexist-encounters-dress-codes-and-demand-for-husbands-involvement-10581211.html

Summary:

Sexism is unfortunately still prevalent in the business world, even for successful entrepreneurs like Vineeta Singh. Read: Shark Tank India's Vineeta Singh on normalising crying at work: 'I cry a lot'
“I have had various types of sexist encounters starting with the fact that I was told we don’t invest in women founded companies. Discover the latest business news, Sensex, and Nifty updates.

Full Text Article:

Sexism is unfortunately still prevalent in the business world, even for successful entrepreneurs like Vineeta Singh. In a recent interview with actor Kareena Kapoor Khan on her show “What Women Want”, the Shark Tank India judge and SUGAR Cosmetics CEO opened up about some of the sexist encounters she has faced early in her career.
The entrepreneur shared an incident where she was asked to dress differently in order to secu

''